<a href="https://colab.research.google.com/github/slowandfast/DiagRecommdSystem_01/blob/main/KJH_Doc2Vec_EMR%EA%B8%B0%EB%B0%98_%EC%A7%84%EB%8B%A8%EC%B6%94%EC%B2%9C%EB%AA%A8%EB%8D%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [83]:
import pandas as pd
from pandas import Series, DataFrame
import gensim
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
import re
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
file_path = '/content/신장내과_본원_외래_20230701_20231231_filtered.xlsx'

selected_columns = ['ICD10', 'transstr']
df = pd.read_excel(file_path, usecols=selected_columns)

print('전체 데이터 수 :',len(df))
df.head(5)

전체 데이터 수 : 826


,ICD10,transstr
0,N391,115-61-61 2023.5.10.Suspicion of proteinuria i...
1,N184,"130/61/69 Symptoms (-), edema (-), urbanic sym..."
2,N184,"153/75/85 osophy (-), edema (-), urinary sympt..."
3,N185,148/90/64 Idema Mild CKD D/T ADPKD has been tr...
4,N183,"Hello teacher, I am a patient who is being hos..."


In [5]:
from sklearn.model_selection import train_test_split

# 학습용, 테스트용, 검증용 데이터 분할
X_train, X_temp, Y_train, Y_temp = train_test_split(df['transstr'], df['ICD10'], test_size=0.2, random_state=5)
X_test, X_val, Y_test, Y_val = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=7)

print(X_train)
print("=======================================\n")
print(X_test)
print("=======================================\n")
print(X_val)
print("=======================================\n")
print(Y_train)
print("=======================================\n")
print(Y_test)
print("=======================================\n")
print(Y_val)

443    2021.09 CABG surgery 2023.06 CR 4.0, EGFR 14 2...
441    Since 2016, Hematuria had hematuria in 2022 TY...
66     23.04 LMC obstetrics and gynecology visits Hem...
773    60s HTN Diagnosis 2023.11.21 CR 1.58 BP 121/67...
131    I didn't have a previous checkup.2023. CR 1.83...
                             ...                        
73     156/94/83, HOME BP <130/80, and Type A told fo...
400    # HTN, On Medication.Comprehensive Award at th...
118    The patient is a patient of 72 years old, curr...
701    HTN, DM 25 -year LMC Drug During LMC Pharmaceu...
206    The patient is a 69 -year -old (m) patient cur...
Name: transstr, Length: 660, dtype: object

278    114-69-88 2 years ago Urology and Browno Holy ...
690    97-53-112 Diabetes and Hypertension-more than ...
108    145-75-85 2023.4.Microscopic Hematuria 2022.10...
499    2023/04 I heard that there is proteinuria duri...
755    The patient is a patient of 77 years old, curr...
                             ...            

In [1]:
pip install googletrans==4.0.0-rc1

In [2]:
!git clone https://github.com/rkiee/project-emr2kcd-dataset.git

Cloning into 'project-emr2kcd-dataset'...
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 38 (delta 14), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (38/38), 3.63 MiB | 9.92 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [6]:
# stopwords dataset load
sw_path = "/content/project-emr2kcd-dataset/stopwords.xlsx"
sw_df = pd.read_excel(sw_path)
for i in range(len(sw_df)) :
  stop_words = sw_df['stop words'].tolist()
print(stop_words)

['A', 'about', 'and', 'at', 'B', 'bid', 'by', 'c', 'cm', 'D', 'd', 'day', 'E', 'F', 'f', 'G', 'g', 'h', 'home', 'i', 'kg', 'known', 'L', 'M', 'mg', 'min', 'ml', 'mm', 'o', 'O', 'of', 'on', 'or', 'P', 'p', 'qd', 'qid', 'R', 'r', 'S', 's', 'T', 't', 'tid', 'U', 'u', 'undefined', 'W', 'w', 'with', 'x', 'y', 'yr', 'yrs', '가끔', '가능', '가능성', '가도', '가렵', '가운데', '가톨릭', '가톨릭대', '간의', '감기', '감사합니다', '갑자기', '강기훈', '강남', '강릉', '강사', '강심', '개', '개월', '개월전', '거의', '거주', '거창', '건강', '검사', '검진', '것', '결과', '결명자', '경', '경기', '경남', '경대', '경대수', '경북', '경산', '경상북도', '경우', '경운기', '경주', '경향', '경희대', '계대', '계란찜', '계명대', '계속', '고견', '고등학교', '고려대련', '고령', '고자', '고진', '고함', '공대', '공주의료원', '과', '과거', '과초', '곽병원', '곽진호', '관리', '관찰', '교수', '교차', '구', '구미', '구미차병원', '국적', '권유', '귀과', '금', '금번', '금일', '기능', '기독', '기본', '김광원', '김기남', '김기호', '김남균', '김완욱', '김준성', '김천', '김현철', '나사렛', '나은병원', '년도', '년전', '니', '다른', '다리', '다시', '다음', '달', '달동', '달마', '당시', '당일', '대가대', '대경영상의학과', '대구', '대구가톨릭대학교', '대남', '대하', '대한', '대해', 

In [17]:
def remove_nonAscii(text):
    rtn = "".join(str for str in text if ord(str)<128)
    return rtn

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    rtn = text.split()
    rtn = [w for w in rtn if not w in stop_words]
    rtn = " ".join(rtn)
    return rtn

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    rtn = tokenizer.tokenize(text)
    rtn = " ".join(rtn)
    return rtn

def remove_others(text):
    s_text = text.split()
    s_list = []
    for w in s_text:
        if len(w) > 1:
            s_list.append(w)
    rtn = " ".join(s_list)
    return rtn


X_train_cleaned = X_train.apply(remove_nonAscii)
X_train_cleaned = X_train_cleaned.apply(make_lower_case)
X_train_cleaned = X_train_cleaned.apply(remove_stop_words)
X_train_cleaned = X_train_cleaned.apply(remove_punctuation)
X_train_cleaned = X_train_cleaned.apply(remove_others)

X_test_cleaned = X_test.apply(remove_nonAscii)
X_test_cleaned = X_test_cleaned.apply(make_lower_case)
X_test_cleaned = X_test_cleaned.apply(remove_stop_words)
X_test_cleaned = X_test_cleaned.apply(remove_punctuation)
X_test_cleaned = X_test_cleaned.apply(remove_others)

X_val_cleaned = X_val.apply(remove_nonAscii)
X_val_cleaned = X_val_cleaned.apply(make_lower_case)
X_val_cleaned = X_val_cleaned.apply(remove_stop_words)
X_val_cleaned = X_val_cleaned.apply(remove_punctuation)
X_val_cleaned = X_val_cleaned.apply(remove_others)

In [8]:
print(X_train_cleaned)

443    cabg surgery cr egfr cr egfr seoul samsung hos...
441    since hematuria had hematuria in type dm diagn...
66     lmc obstetrics gynecology visits hematuria con...
773                                  htn diagnosis cr bp
131    didn have previous checkup cr is confirmed vis...
                             ...                        
73       bp type told for kidney donation symptoms edema
400    htn medication comprehensive award the health ...
118    the patient is patient years old currently bei...
701    htn dm year lmc drug during lmc pharmaceutical...
206    the patient is year old patient currently bein...
Name: transstr, Length: 660, dtype: object


In [11]:
# corpus = []

# for words in X_train_cleaned:
#     corpus.append(words.split())

# print(corpus[:])

[['cabg', 'surgery', 'cr', 'egfr', 'cr', 'egfr', 'seoul', 'samsung', 'hospital', 'professor', 'said', 'that'], ['since', 'hematuria', 'had', 'hematuria', 'in', 'type', 'dm', 'diagnosis', 'cr', 'egfr', 'cr', 'egfr', 'bp', 'bw'], ['lmc', 'obstetrics', 'gynecology', 'visits', 'hematuria', 'confirmed', 'when', 'inspected', 'dysuria'], ['htn', 'diagnosis', 'cr', 'bp'], ['didn', 'have', 'previous', 'checkup', 'cr', 'is', 'confirmed', 'visited', 'from', 'the', 'test', 'conducted', 'due', 'to', 'increased', 'blood', 'pressure', 'did', 'not', 'implement', 'bubble', 'urine', 'health', 'supplement'], ['it', 'has', 'been', 'edema', 'since', 'childhood', 'cr', 'tft', 'normal', 'serum', 'osm', 'na'], ['htn', 'cr', 'egfr', 'hba', 'na'], ['ginseng', 'antler', 'have', 'been', 'taking', 'two', 'years', 'ago', 'operation', 'gangwon', 'bun', 'cr', 'egfr', 'bp'], ['in', 'high', 'school', 'examination', 'hematururia', 'proteinuria', 'finds', 'from', 'the', 'military', 'service', 'administration', 'bp'], ['c

In [13]:
# maxlen = 0
# maxidx = -1

# for i in range(len(corpus)) :
#     if maxlen < len(corpus[i]) :
#         maxlen = len(corpus[i])
#         maxidx = i

# print("max row length : ", maxlen)
# print("max row index : ", maxidx)
# print("max row contents : ", corpus[maxidx])
# print("max row original contents : ", X_train_cleaned.iloc[maxidx])

max row length :  147
max row index :  497
max row contents :  ['cerebral', 'infarction', 'cr', 'pp', 'mca', 'proximal', 'occlusion', 'iat', 'ivt', 'essential', 'atrial', 'fibrillation', 'scc', 'lt', 'cheek', 'pt', 'pnx', 'fatima', 'excision', 'rtx', 'hematoma', 'hematuria', 'anemia', 'dimer', 'elevation', 'ckd', 'uti', 'crcl', 'ml', 'min', 'wt', 'kg', 'cr', 'risk', 'hba', 'tc', 'ldl', 'tg', 'tte', 'la', 'cm', 'ef', 'rwma', 'mild', 'mod', 'mr', 'mild', 'mod', 'tr', 'mod', 'severe', 'as', 'pfo', 'tte', 'lmc', 'la', 'cm', 'ef', 'rwma', 'mild', 'modmhe', 'is', 'hospitalized', 'the', 'patient', 'name', 'elevation', 'is', 'confirmed', 'bun', 'mg', 'dl', 'creatinine', 'mg', 'dl', 'in', 'the', 'lab', 'which', 'was', 'carried', 'out', 'patient', 'was', 'confirmed', 'lab', 'lmc', 'fatima', 'hospital', 'requested', 'the', 'patient', 'ckd', 'checked', 'the', 'answer', 'used', 'antibiotics', 'for', 'the', 'uti', 'the', 'abdominal', 'usg', 'was', 'identified', 'as', 'right', 'kidney', 'cm', 'left',

In [35]:
print(Y_train)

443    N185
441    N183
66     R311
773    N183
131    N183
       ... 
73     Z524
400    R311
118    N183
701    N184
206    N183
Name: ICD10, Length: 660, dtype: object


In [46]:
# Doc2Vec 모델 학습을 위해 학습 데이터 포맷 변환 (TaggedDocument 객체 생성)
train_tagged_data = []
row = []
idx = -1
for words in X_train_cleaned :
    row.append(words.split())
    idx += 1
    # if i in range(5):
    #     print("word corpus : ", corpus[i])
    #     print("Y_train : ", Y_train.iloc[i])
    train_tagged_data.append(TaggedDocument(words=row[idx], tags=Y_train.iloc[idx]))


In [68]:
# Doc2Vec 모델 테스트를 위해 테스트 데이터 포맷 변환 (TaggedDocument 객체 생성)
test_tagged_data = []
row = []
idx = -1
for words in X_test_cleaned :
    row.append(words.split())
    idx += 1
    test_tagged_data.append(TaggedDocument(words=row[idx], tags=Y_train.iloc[idx]))

In [42]:
# 태그 확인
for words, tags in train_tagged_data:
    print(f"tags: {tags}, words: {words}")

tags: N185, words: ['cabg', 'surgery', 'cr', 'egfr', 'cr', 'egfr', 'seoul', 'samsung', 'hospital', 'professor', 'said', 'that']
tags: N183, words: ['since', 'hematuria', 'had', 'hematuria', 'in', 'type', 'dm', 'diagnosis', 'cr', 'egfr', 'cr', 'egfr', 'bp', 'bw']
tags: R311, words: ['lmc', 'obstetrics', 'gynecology', 'visits', 'hematuria', 'confirmed', 'when', 'inspected', 'dysuria']
tags: N183, words: ['htn', 'diagnosis', 'cr', 'bp']
tags: N183, words: ['didn', 'have', 'previous', 'checkup', 'cr', 'is', 'confirmed', 'visited', 'from', 'the', 'test', 'conducted', 'due', 'to', 'increased', 'blood', 'pressure', 'did', 'not', 'implement', 'bubble', 'urine', 'health', 'supplement']
tags: R601, words: ['it', 'has', 'been', 'edema', 'since', 'childhood', 'cr', 'tft', 'normal', 'serum', 'osm', 'na']
tags: N184, words: ['htn', 'cr', 'egfr', 'hba', 'na']
tags: N178, words: ['ginseng', 'antler', 'have', 'been', 'taking', 'two', 'years', 'ago', 'operation', 'gangwon', 'bun', 'cr', 'egfr', 'bp']
ta

In [91]:
# Doc2Vec 모델 생성 및 학습
model = Doc2Vec(vector_size=20, window = 1, min_count=1, epochs=50)
# 모델에 문서 집합을 토큰화하여 단어장을 생성
model.build_vocab(train_tagged_data)
# 학습
model.train(train_tagged_data, total_examples=model.corpus_count, epochs=model.epochs)
# 모델 저장
# model.save("/content/d2v.model")

In [92]:
print(X_train_cleaned.shape)

(660,)


In [96]:
# 각 진단코드 별 문서 벡터의 평균을 계산
title_vectors = {}

# 전체 목록을 돌면서
# ICD10 기준으로 나눔
for words, tags in train_tagged_data:
    vector = model.infer_vector(words)

    if tags not in title_vectors:
        title_vectors[tags] = [vector]
    else:
        title_vectors[tags].append(vector)


In [ ]:
# print(title_vectors["Z524"])
for k, v in title_vectors.items():
    print("key [%s] => value [%s]" % (k, v))

In [64]:
newv = [[ 0.05191375, -0.35807624, -1.1172268 , -0.3456536 ,  1.3029747 ,
       -0.6923046 , -1.3339084 , -1.7402083 ,  0.54769224, -1.2900263 ,
        0.06877777,  0.75874627, -1.6898298 , -1.4068799 , -0.6595111 ,
        0.40042508, -0.6698096 , -0.1028536 , -1.7230858 ,  0.9031247 ],
      [-0.5320899 , -0.55325913, -0.56734025, -0.44084147,  1.8470918 ,
       -2.5460072 , -3.2236536 , -2.4175048 , -0.28826147, -3.9946704 ,
        0.12297299,  0.40201938, -3.1063411 , -3.1591284 , -2.9409873 ,
        0.3978219 , -1.8036907 , -0.64685625, -3.6773157 ,  2.1172113 ]]


meanv = np.mean(newv, axis=0)

print(meanv)

[-0.24008808 -0.45566768 -0.84228353 -0.39324753  1.57503325 -1.6191559
 -2.278781   -2.07885655  0.12971539 -2.64234835  0.09587538  0.58038282
 -2.39808545 -2.28300415 -1.8002492   0.39912349 -1.23675015 -0.37485493
 -2.70020075  1.510168  ]


In [97]:
# 같은 레이블의 평균 벡터 계산
averaged_documents = []
for title, vectors in title_vectors.items():
    averaged_vector = np.mean(vectors, axis=0)
    averaged_documents.append((title, averaged_vector))

# print(averaged_documents)
# print(len(averaged_documents))

# 평균화된 벡터를 사용하여 모델 재학습
for title, vector in averaged_documents:
    # print(title)
    model.dv[title] = vector

In [101]:
print(X_test_cleaned)
print(Y_test)

278    years ago urology browno holy hematopathy use ...
690    diabetes hypertension more than years peripher...
108    microscopic hematuria after hematuria proteinu...
499    heard that there is proteinuria during examina...
755    the patient is patient years old currently bei...
                             ...                        
228    the patient is diagnostic name paralysis vocal...
151    htn cr egfr egfr na na onion juice gun gangwon...
129    the patient is diagnostic djd patient years ol...
781    we will ask for consultation the patient is ye...
159    years ago dm htn diagnosis years ago renal fun...
Name: transstr, Length: 83, dtype: object
278    R311
690    N183
108    R311
499    R311
755    N183
       ... 
228    N183
151    N183
129    N183
781    R311
159    N183
Name: ICD10, Length: 83, dtype: object


In [103]:
print(df['transstr'].iloc[690])

97-53-112 Diabetes and Hypertension-more than 15 years of peripheral polyneuropathy + stroke (CBR HMRR) + -RT Hemiplegia Appetite Urginet and Creainine 1.43 UA Prot + RBC 40-50/HPF


In [ ]:
for words, tags in test_tagged_data:
    print(f"tags: {tags}, words: {words}")

In [104]:
# 입력된 내용과 유사한 문서 검색
input_text = input("현병력을 입력하세요: ")
input_list = [1]
input_list[0] = input_text

input_df = Series(input_list)
print(input_df)
# X_input_cleaned = remove_nonAscii(input_list)
# X_input_cleaned = make_lower_case(X_input_cleaned)
# X_input_cleaned = remove_stop_words(X_input_cleaned)
# X_input_cleaned = remove_punctuation(X_input_cleaned)
# X_input_cleaned = remove_others(X_input_cleaned)

X_input_cleaned = input_df.apply(remove_nonAscii)
X_input_cleaned = X_input_cleaned.apply(make_lower_case)
X_input_cleaned = X_input_cleaned.apply(remove_stop_words)
X_input_cleaned = X_input_cleaned.apply(remove_punctuation)
X_input_cleaned = X_input_cleaned.apply(remove_others)

print("1 >> ", X_input_cleaned)

input_tokens = []

for words in X_input_cleaned:
    input_tokens.append(words.split())

print("2 >> ", input_tokens)
input_vector = model.infer_vector(input_tokens[0])

similar_documents = model.dv.most_similar([input_vector], topn=5)

for title, score in similar_documents:
    print(f"{title} similarity: {score:.3f}")


현병력을 입력하세요: Diabetes and Hypertension-more than 15 years of peripheral polyneuropathy + stroke (CBR HMRR) + -RT Hemiplegia Appetite Urginet and Creainine 1.43 UA Prot + RBC 40-50/HPF
0    Diabetes and Hypertension-more than 15 years o...
dtype: object
1 >>  0    diabetes hypertension more than years peripher...
dtype: object
2 >>  [['diabetes', 'hypertension', 'more', 'than', 'years', 'peripheral', 'polyneuropathy', 'stroke', 'cbr', 'hmrr', 'rt', 'hemiplegia', 'appetite', 'urginet', 'creainine', 'ua', 'prot', 'rbc', 'hpf']]


ValueError: operands could not be broadcast together with shapes (27,) (13,) 

In [ ]:
print(X_test)
print("\n")
print(Y_test)

168    The patient is a patient of 55 -year -old F, c...
836    155/73/91 yotrophic symptoms (-), edema (-), u...
76     The patient is a 92 -year -old (m) patient cur...
70     2019. CR 1.33 EGFR 39 2022. CR 1.3 EGFR 39 U/A...
88     When urinated, it is fine, but the residue is ...
                             ...                        
965    The patient is the 65 -year -old (F) patient c...
633    The patient is commissioned by AZOTEMIA, a gua...
557    2023.05.03 CAG DM, HTN for 20 years or more 20...
33     #Brainstem Infarction, LT PONS #LT MCA M1 foca...
31     I have never seen HTN (25 years ago) DM (25 ye...
Name: transstr, Length: 195, dtype: object


168    N183
836    N183
76     N183
70     N183
88     N390
       ... 
965    N183
633    N178
557    N183
33     R609
31     N183
Name: ICD10, Length: 195, dtype: object


In [ ]:
X_test_cleaned = X_test.apply(remove_nonAscii)
X_test_cleaned = X_test_cleaned.apply(make_lower_case)
X_test_cleaned = X_test_cleaned.apply(remove_stop_words)
X_test_cleaned = X_test_cleaned.apply(remove_punctuation)
X_test_cleaned = X_test_cleaned.apply(remove_others)

print(X_test_cleaned)

168    the patient is patient year old currently bein...
836    yotrophic symptoms edema urbanic symptoms doe ...
76     the patient is year old patient currently bein...
70                               cr egfr cr egfr cr egfr
88     when urinated it is fine but the residue is se...
                             ...                        
965    the patient is the year old patient currently ...
633    the patient is commissioned azotemia guardian ...
557    cag dm htn for years more the cag diocese in t...
33     brainstem infarction lt pons lt mca focal sten...
31     have never seen htn years ago dm years ago cva...
Name: transstr, Length: 195, dtype: object


In [ ]:
print(Y_test.iloc[0])
print(X_test_cleaned.iloc[0])

N183
the patient is patient year old currently being treated diagnostic name darier disease the diagnostic name is confirmed the valtrex neotigasone confirmed as the lab phase bun cr which is performed in the main department is required to be required please be busy derma prof vs seoul daegu ointment seoul had drug in kidney proteinuria at the time blood pressure medicine vitamin omega the time pharmaceuticals in personal psychiatric clinic can sleep during the medication for year depression is severe


In [ ]:
test_corpus = []

for words in X_test_cleaned:
    test_corpus.append(words.split())

print(test_corpus[:])

[['the', 'patient', 'is', 'patient', 'year', 'old', 'currently', 'being', 'treated', 'diagnostic', 'name', 'darier', 'disease', 'the', 'diagnostic', 'name', 'is', 'confirmed', 'the', 'valtrex', 'neotigasone', 'confirmed', 'as', 'the', 'lab', 'phase', 'bun', 'cr', 'which', 'is', 'performed', 'in', 'the', 'main', 'department', 'is', 'required', 'to', 'be', 'required', 'please', 'be', 'busy', 'derma', 'prof', 'vs', 'seoul', 'daegu', 'ointment', 'seoul', 'had', 'drug', 'in', 'kidney', 'proteinuria', 'at', 'the', 'time', 'blood', 'pressure', 'medicine', 'vitamin', 'omega', 'the', 'time', 'pharmaceuticals', 'in', 'personal', 'psychiatric', 'clinic', 'can', 'sleep', 'during', 'the', 'medication', 'for', 'year', 'depression', 'is', 'severe'], ['yotrophic', 'symptoms', 'edema', 'urbanic', 'symptoms', 'doe', 'dm', 'yrs', 'htn', 'yrs', 'patient', 'lmc', 'inspectionhba', 'prot', 'visit'], ['the', 'patient', 'is', 'year', 'old', 'patient', 'currently', 'being', 'treated', 'under', 'the', 'diagnosti

In [ ]:
test_tagged_data = []

for i in range(len(test_corpus)) :
    test_tagged_data.append(TaggedDocument(words=test_corpus[i], tags=Y_test.iloc[i]))

In [ ]:
print(test_tagged_data[0])
print(test_tagged_data[1])

TaggedDocument<['the', 'patient', 'is', 'patient', 'year', 'old', 'currently', 'being', 'treated', 'diagnostic', 'name', 'darier', 'disease', 'the', 'diagnostic', 'name', 'is', 'confirmed', 'the', 'valtrex', 'neotigasone', 'confirmed', 'as', 'the', 'lab', 'phase', 'bun', 'cr', 'which', 'is', 'performed', 'in', 'the', 'main', 'department', 'is', 'required', 'to', 'be', 'required', 'please', 'be', 'busy', 'derma', 'prof', 'vs', 'seoul', 'daegu', 'ointment', 'seoul', 'had', 'drug', 'in', 'kidney', 'proteinuria', 'at', 'the', 'time', 'blood', 'pressure', 'medicine', 'vitamin', 'omega', 'the', 'time', 'pharmaceuticals', 'in', 'personal', 'psychiatric', 'clinic', 'can', 'sleep', 'during', 'the', 'medication', 'for', 'year', 'depression', 'is', 'severe'], N183>
TaggedDocument<['yotrophic', 'symptoms', 'edema', 'urbanic', 'symptoms', 'doe', 'dm', 'yrs', 'htn', 'yrs', 'patient', 'lmc', 'inspectionhba', 'prot', 'visit'], N183>


In [ ]:
# 테스트 데이터 태그 확인
for words, tags in test_tagged_data:
    print(f"tags: {tags}, words: {words}")

In [ ]:
for text in test_corpus:
    inferred_v = model.infer_vector(text)

    print(f"vector of {text}: {inferred_v}")

    most_similar_docs = model.docvecs.most_similar([inferred_v], topn=5)

    for index, similarity in most_similar_docs:
        print(f"{index}, similarity: {similarity}")

    print(most_similar_docs)
    print("=="*20)

text :  ['the', 'patient', 'is', 'patient', 'year', 'old', 'currently', 'being', 'treated', 'diagnostic', 'name', 'darier', 'disease', 'the', 'diagnostic', 'name', 'is', 'confirmed', 'the', 'valtrex', 'neotigasone', 'confirmed', 'as', 'the', 'lab', 'phase', 'bun', 'cr', 'which', 'is', 'performed', 'in', 'the', 'main', 'department', 'is', 'required', 'to', 'be', 'required', 'please', 'be', 'busy', 'derma', 'prof', 'vs', 'seoul', 'daegu', 'ointment', 'seoul', 'had', 'drug', 'in', 'kidney', 'proteinuria', 'at', 'the', 'time', 'blood', 'pressure', 'medicine', 'vitamin', 'omega', 'the', 'time', 'pharmaceuticals', 'in', 'personal', 'psychiatric', 'clinic', 'can', 'sleep', 'during', 'the', 'medication', 'for', 'year', 'depression', 'is', 'severe']
vector of ['the', 'patient', 'is', 'patient', 'year', 'old', 'currently', 'being', 'treated', 'diagnostic', 'name', 'darier', 'disease', 'the', 'diagnostic', 'name', 'is', 'confirmed', 'the', 'valtrex', 'neotigasone', 'confirmed', 'as', 'the', 'lab'

<ipython-input-78-f0ed9bdf42c4>:7: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  most_similar_docs = model.docvecs.most_similar([inferred_v], topn=5)



text :  ['bw', 'kg', 'edema', 'doe', 'symptoms', 'jjaesim', 'htn', 'hypothyroidism', 'patients', 'in', 'it', 'has', 'been', 'edema', 'since', 'the', 'city', 'but', 'has', 'been', 'worse', 'for', 'the', 'last', 'two', 'months', 'even', 'if', 'you', 'take', 'dichhl', 'aldacton', 'bids', 'you', 'can', 'continue', 'to', 'edema', 'cr', 'in', 'lmc', 'inspection']
vector of ['bw', 'kg', 'edema', 'doe', 'symptoms', 'jjaesim', 'htn', 'hypothyroidism', 'patients', 'in', 'it', 'has', 'been', 'edema', 'since', 'the', 'city', 'but', 'has', 'been', 'worse', 'for', 'the', 'last', 'two', 'months', 'even', 'if', 'you', 'take', 'dichhl', 'aldacton', 'bids', 'you', 'can', 'continue', 'to', 'edema', 'cr', 'in', 'lmc', 'inspection']: [ 1.632144    0.9347799  -0.8426718  -0.36838245  1.267835   -1.1680372
  0.5413485  -1.0587097   1.5689474  -2.6070957  -1.3244075   1.6546031
 -3.5188453  -3.4907553  -2.418596    3.2333872  -1.5332375  -1.0474379
  0.78121316  1.8639047 ]
C, similarity: 0.8115847110748291


In [ ]:
y_true = [] # 실제값
y_pred = [] # 예측값

for i, row in X_test_cleaned:
    text = X_test_cleaned.iloc[i]

    input_tokens = text
    # print(input_tokens)

    input_vector = model.infer_vector(input_tokens) # 모델에 입력되는 벡터를 추론

    # X_train의 각 문서에 대해 입력벡터와의 유사도를 계산
    similarities = []
    for j, row_train  in X_train.iterrows():
        # j번째 문서에 대한 docment vector
        # document_vector = model.dv[row_train['DIAGCD']]
        document_vector = model.dv[row_train['ICD10']]
        # input_vector와 document_vector 사이의 유사도 계산
        similarity = cosine_similarity([input_vector], [document_vector])[0][0]
        # # 계산된 유사도를 리스트에 추가
        similarities.append(similarity)

    # 가장 높은 유사도를 가진 doc의 인덱스를 가져온다
    predicted_index = np.argmax(similarities)
    # print(predicted_index)
    # 실제값을 y_true 리스트에 추가하고, 예측값을  y_pred 리스트에 추가
    # y_true.append(row['DIAGCD'])
    # y_pred.append(X_train.iloc[predicted_index]['DIAGCD'])
    y_true.append(row['ICD10'])
    y_pred.append(X_train.iloc[predicted_index]['ICD10'])

print(f"예측값 : {y_pred}")
print(f"실제값 : {y_true}\n")

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted', zero_division='warn')
f1 = f1_score(y_true, y_pred, average='weighted', zero_division='warn')
recall = recall_score(y_true, y_pred, average='weighted', zero_division='warn')

print(f"Accuracy: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"F1 Score: {f1:.3f}")
print(f"Recall: {recall:.3f}")

AttributeError: 'Series' object has no attribute 'iterrows'